In [55]:
# Program To Read video 
# and Extract Frames 

# import sys
# !{sys.executable} -m pip install opencv-python
# !{sys.executable} -m pip install moviepy

import cv2 
import os
import sys
import shutil
import moviepy.video.io.ImageSequenceClip

In [46]:
def FrameCapture(path, vid_num): 
      
    # Path to video file 
    vidObj = cv2.VideoCapture(path) 
  
    # Used as counter variable 
    count = 0
    success = 1
    images=[]
    try:
        shutil.rmtree("lebron_dunk" + str(vid_num))
        os.mkdir("lebron_dunk" + str(vid_num))
    except OSError as e:
        os.mkdir("lebron_dunk" + str(vid_num))
    while success: 
  
        # vidObj object calls read 
        # function extract frames 
        success, image = vidObj.read() 
        # Saves the frames with frame-count 
        if image is not None and len(image) > 0:
            images.append(image)
            cv2.imwrite("lebron_dunk" + str(vid_num) + "/frame" + str(1000+count) + ".jpg", image) 
#                cv2.imwrite("frame%d.jpg" % count, image)
        count += 1
    return images

In [47]:
# for i in range(1, 11):
#     FrameCapture("movies/lebron_dunk" + str(i) + ".mp4", i)

In [60]:
def recreate_movie(image_folder, new_name):
    fps=30
    image_files = [image_folder+'/'+img for img in os.listdir(image_folder) if img.endswith(".jpg")]
    image_files.sort()
    clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=fps)
    clip.write_videofile(new_name + '.mp4')

In [61]:
recreate_movie('lebron_dunk1', 'lebron_1')

t:   8%|▊         | 9/119 [00:00<00:01, 87.24it/s, now=None]

Moviepy - Building video lebron_1.mp4.
Moviepy - Writing video lebron_1.mp4



Moviepy - Done !
Moviepy - video ready lebron_1.mp4


In [ ]:
import torch 
import numpy as np

In [ ]:
from torchvision import datasets, transforms

## YOUR CODE HERE ##
batch_size = 10
latent_dim = 10
transformations = transforms.ToTensor()
# mnist_train =  datasets.MNIST(loc_data, train=True, download=True, transform = transformations)
# mnist_test = datasets.MNIST(loc_data, train=False, download=True, transform = transformations)

In [ ]:
images = FrameCapture("movies/lebron_dunk1.mp4", 1)
np.array(images).shape
total_len = 540*960

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(images,batch_size=batch_size,drop_last = True,shuffle=False)

In [ ]:
from torch import nn
class Encoder(nn.Module):
  def __init__(self, latent_dim):
    super(Encoder, self).__init__()
    ## YOUR CODE HERE ##
    self.lin1 = nn.Linear(total_len, total_len/2)
    self.lin2 = nn.Linear(total_len/2, total_len/4)
    self.lin3 = nn.Linear(total_len/4, total_len/10)
    self.lin4a = nn.Linear(total_len/10, latent_dim)
    self.lin4b = nn.Linear(total_len/10, latent_dim)
  
  def forward(self, x):
    ## YOUR CODE HERE ##\
    x = x.view(-1, total_len)
    x = nn.functional.relu(self.lin1(x))
    x = nn.functional.relu(self.lin2(x))
    x = nn.functional.relu(self.lin3(x))
    # mu, log_var
    return self.lin4a(x), self.lin4b(x)

class Decoder(nn.Module):
  def __init__(self, latent_dim):
    super(Decoder, self).__init__()
    ## YOUR CODE HERE ##
    self.lin1 = nn.Linear(latent_dim, total_len/10)
    self.lin2 = nn.Linear(total_len/10, total_len/4)
    self.lin3 = nn.Linear(total_len/4, total_len/2)
    self.lin4 = nn.Linear(total_len/2, total_len)

  def forward(self,x):
    ## YOUR CODE HERE ##
    x = nn.functional.relu(self.lin1(x))
    x = nn.functional.relu(self.lin2(x))
    x = nn.functional.relu(self.lin3(x))
    x = torch.sigmoid(self.lin4(x))
    return x.view(-1, 540, 960)